# 01 - DATA PREPROCESSING

## Executive Summary

This notebook cleans and preprocesses the **Agrofood CO₂ emissions** dataset for use in later modeling and policy scenario analysis.

- **Source:** `Agrofood_co2_emission.csv`  
- **Size:** 6,965 rows × 31 columns  
- **Structure:** Country–year panel data (1990–2020) with:
  - Target: `total_emission`
  - Temperature: `Average Temperature °C`
  - Emission/activity drivers (e.g., `Crop Residues`, `Forest fires`, `Forestland`, `On-farm energy use`, `IPPU`, etc.)
  - Demographics: `Rural population`, `Urban population`, `Total Population - Male/Female`

***Key Cleaning Steps:***

1. **Integrity checks**
   - Verified that (`Area`, `Year`) uniquely identifies each record (no duplicates).
   - Set `Area` as categorical and ensured `Year` is integer.

2. **Missing data handling**
   - Diagnosed missingness: concentrated in a few emission variables (up to ~20%), none in target or population variables.
   - Kept all rows (max row-wise missingness ≈ 29%).
   - Imputation (for all numeric columns except `Year`):
     - Within each country, interpolated over `Year`.
     - Filled remaining NAs with country-wise means.
     - Final fallback to global medians.
   - Confirmed **0 missing values** in `df_clean`.

3. **Sanity checks**
   - Examined negative values:
     - `Forestland`, `total_emission`, and some temperatures can be legitimately negative (net sinks, cold years) and are retained.
   - Checked min–max ranges; large population values for India/China are plausible.

4. **Output**
   - Saved cleaned dataset as `Agrofood_co2_emission_clean.csv`, which will be the starting point for all subsequent EDA, modeling, and scenario analysis.


## Table of Contents

1. [Load and basic inspection](#1-load-and-basic-inspection)
2. [Check duplicates on (Area, Year)](#2-check-duplicates-on-area-year)
3. [Set clean dtypes](#3-set-clean-dtypes)
4. [Missing values – what’s actually missing?](#4-missing-values--whats-actually-missing)
5. [Imputation strategy (time + country aware)](#5-imputation-strategy-time--country-aware)  
    5.1. [Identify numeric columns](#51-identify-numeric-columns)  
    5.2. [Interpolate within each country over time](#52-interpolate-within-each-country-over-time)  
    5.3. [Fill remaining gaps with country means then global medians](#53-fill-remaining-gaps-with-country-means-then-global-medians)  
6. [Sanity checks: negatives and impossible values](#6-sanity-checks-negatives-and-impossible-values)  
    6.1. [Negative counts](#61-negative-counts)  
    6.2. [Optional: Filtered DataFrame](#62-optional-filtered-dataframe)  
7. [Optional: simple range sanity](#7-optional-simple-range-sanity)
8. [Save the cleaned dataset](#8-save-the-cleaned-dataset)

### 1. Load and basic inspection

Optional: For Google Colab

In [ ]:
# import os
# from google.colab import files

# # Upload file
# uploaded = files.upload()

# for filename, file_content in uploaded.items():
#     # If file already exists, delete it to avoid duplicates
#     if os.path.exists(filename):
#         os.remove(filename)
#         print(f"Existing file '{filename}' found and overwritten.")

#     # Write uploaded file to disk
#     with open(filename, 'wb') as f:
#         f.write(file_content)
#         print(f"Uploaded and saved file: {filename}")


In [ ]:
import pandas as pd
import numpy as np

# Load raw
df = pd.read_csv("Agrofood_co2_emission.csv")

# For Google Colab users, uncomment the following line to load the uploaded file
# df = pd.read_csv("Agrofood_co2_emission.csv")

print(df.shape)      # expect (6965, 31)
print(df.columns)    # 31 columns
df.head()
df.info()


(6965, 31)
Index(['Area', 'Year', 'Savanna fires', 'Forest fires', 'Crop Residues',
       'Rice Cultivation', 'Drained organic soils (CO2)',
       'Pesticides Manufacturing', 'Food Transport', 'Forestland',
       'Net Forest conversion', 'Food Household Consumption', 'Food Retail',
       'On-farm Electricity Use', 'Food Packaging',
       'Agrifood Systems Waste Disposal', 'Food Processing',
       'Fertilizers Manufacturing', 'IPPU', 'Manure applied to Soils',
       'Manure left on Pasture', 'Manure Management', 'Fires in organic soils',
       'Fires in humid tropical forests', 'On-farm energy use',
       'Rural population', 'Urban population', 'Total Population - Male',
       'Total Population - Female', 'total_emission',
       'Average Temperature °C'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965 entries, 0 to 6964
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                      

### 2. Check duplicates on (Area, Year)

Set clear dtypes so later preprocessing is smooth:

In [80]:
# Natural key: each Area-Year should be unique
dup_mask = df.duplicated(subset=['Area', 'Year'])
dup_count = dup_mask.sum()
print("Duplicate Area-Year rows:", dup_count)

# If >0, inspect and then drop if appropriate:
df[dup_mask].head()
# df = df[~dup_mask]


Duplicate Area-Year rows: 0


,Area,Year,Savanna fires,Forest fires,Crop Residues,Rice Cultivation,Drained organic soils (CO2),Pesticides Manufacturing,Food Transport,Forestland,...,Manure Management,Fires in organic soils,Fires in humid tropical forests,On-farm energy use,Rural population,Urban population,Total Population - Male,Total Population - Female,total_emission,Average Temperature °C


`dup_count` is 0, so no action needed.

### 3. Set clean dtypes

In [81]:
df_clean = df.copy()

# Categorical for Area
df_clean['Area'] = df_clean['Area'].astype('category')

# Ensure Year is int
df_clean['Year'] = df_clean['Year'].astype(int)
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965 entries, 0 to 6964
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   Area                             6965 non-null   category
 1   Year                             6965 non-null   int64   
 2   Savanna fires                    6934 non-null   float64 
 3   Forest fires                     6872 non-null   float64 
 4   Crop Residues                    5576 non-null   float64 
 5   Rice Cultivation                 6965 non-null   float64 
 6   Drained organic soils (CO2)      6965 non-null   float64 
 7   Pesticides Manufacturing         6965 non-null   float64 
 8   Food Transport                   6965 non-null   float64 
 9   Forestland                       6472 non-null   float64 
 10  Net Forest conversion            6472 non-null   float64 
 11  Food Household Consumption       6492 non-null   float64 
 12  Food R

### 4. Missing values – what’s actually missing?

In [82]:
na_counts = df_clean.isna().sum().sort_values(ascending=False)
na_frac = (df_clean.isna().mean().sort_values(ascending=False))

print(na_counts)
print(na_frac.head(10))


Crop Residues                      1389
On-farm energy use                  956
Manure Management                   928
Manure applied to Soils             928
IPPU                                743
Forestland                          493
Net Forest conversion               493
Food Household Consumption          473
Fires in humid tropical forests     155
Forest fires                         93
Savanna fires                        31
Drained organic soils (CO2)           0
Rice Cultivation                      0
Area                                  0
Year                                  0
Food Packaging                        0
On-farm Electricity Use               0
Food Retail                           0
Pesticides Manufacturing              0
Food Transport                        0
Food Processing                       0
Fertilizers Manufacturing             0
Agrifood Systems Waste Disposal       0
Fires in organic soils                0
Manure left on Pasture                0


Row-wise missingness is also mild (max ≈ 29% per row), so we don’t need to drop rows:

In [83]:
row_missing_frac = df_clean.isna().mean(axis=1)
row_missing_frac.describe()


count    6965.000000
mean        0.030947
std         0.056064
min         0.000000
25%         0.000000
50%         0.000000
75%         0.032258
max         0.290323
dtype: float64

### 5. Imputation strategy (time + country aware)

Here’s a realistic and explainable approach:

> For each country (`Area`), we interpolate each numeric column over `Year`, then fill any remaining gaps with country-specific means, and finally with global medians if still missing.

#### 5.1. Identify numeric columns

In [84]:
# All numeric columns
num_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns

# We don't need to interpolate Year; it's already complete
if 'Year' in num_cols:
    num_cols = num_cols.drop('Year')

num_cols


Index(['Savanna fires', 'Forest fires', 'Crop Residues', 'Rice Cultivation',
       'Drained organic soils (CO2)', 'Pesticides Manufacturing',
       'Food Transport', 'Forestland', 'Net Forest conversion',
       'Food Household Consumption', 'Food Retail', 'On-farm Electricity Use',
       'Food Packaging', 'Agrifood Systems Waste Disposal', 'Food Processing',
       'Fertilizers Manufacturing', 'IPPU', 'Manure applied to Soils',
       'Manure left on Pasture', 'Manure Management', 'Fires in organic soils',
       'Fires in humid tropical forests', 'On-farm energy use',
       'Rural population', 'Urban population', 'Total Population - Male',
       'Total Population - Female', 'total_emission',
       'Average Temperature °C'],
      dtype='object')

This keeps `Year` numeric but excludes it from interpolation/imputation; we mostly care about the emission/population/temperature columns.

#### 5.2. Interpolate within each country over time

In [85]:
# Sort first so interpolation respects time ordering
df_clean = df_clean.sort_values(['Area', 'Year'])

# Interpolate numeric columns within each Area
df_clean[num_cols] = (
    df_clean
      .groupby('Area', observed=False)[num_cols]
      .apply(lambda g: g.interpolate(limit_direction='both'))
      .reset_index(level=0, drop=True)
)
# Verify interpolation
df_clean.isna().sum().sort_values(ascending=False)

Crop Residues                      1227
On-farm energy use                  922
Manure Management                   918
Manure applied to Soils             918
IPPU                                743
Forestland                          493
Net Forest conversion               493
Food Household Consumption          445
Fires in humid tropical forests     155
Forest fires                         93
Savanna fires                        31
Drained organic soils (CO2)           0
Rice Cultivation                      0
Area                                  0
Year                                  0
Food Packaging                        0
On-farm Electricity Use               0
Food Retail                           0
Pesticides Manufacturing              0
Food Transport                        0
Food Processing                       0
Fertilizers Manufacturing             0
Agrifood Systems Waste Disposal       0
Fires in organic soils                0
Manure left on Pasture                0


What this does:
- If Afghanistan has `Crop Residues` for 1990 and 1992 but not 1991, 1991 is filled between those two.
- If a series is missing at the start or end, `limit_direction='both'` extrapolates from the nearest known value in that group.

#### 5.3. Fill remaining gaps with country means then global medians

Interpolation can’t fix cases where entire time ranges are missing for a country-variable pair, so we follow with:

In [86]:
# Fill remaining NAs with Area-wise mean
df_clean[num_cols] = df_clean.groupby('Area', observed=False)[num_cols].transform(
    lambda g: g.fillna(g.mean())
)

# Last resort: global median, column-wise
df_clean[num_cols] = df_clean[num_cols].fillna(df_clean[num_cols].median())

# Check we’re done with missingness
df_clean.isna().sum().sort_values(ascending=False).head()


Area             0
Year             0
Savanna fires    0
Forest fires     0
Crop Residues    0
dtype: int64

In [87]:
# Extra sanity check: make sure absolutely no NaNs remain
total_nas = df_clean.isna().sum().sum()
print("Total remaining NAs:", total_nas)


Total remaining NAs: 0


*"We have verified that the final cleaned dataset has no missing values."*

### 6. Sanity checks: negatives and impossible values

#### 6.1. Negative counts

In [88]:
neg_counts = (df_clean[num_cols] < 0).sum().sort_values(ascending=False)
neg_counts.head(10)


Forestland                     4717
Average Temperature °C          288
total_emission                  238
Forest fires                      0
Crop Residues                     0
Drained organic soils (CO2)       0
Pesticides Manufacturing          0
Food Transport                    0
Net Forest conversion             0
Food Household Consumption        0
dtype: int64

#### 6.2. Optional: Filtered DataFrame

- **Forestland**: Negatives are expected (net loss/deforestation or net sink). We will keep them.
- **Average Temperature °C**: Negative temperatures values are standard. They will be kept.
- **total_emission:**
    - Negative values likely represent net sinks (more removal than emission).
    - For cleaning, We can keep them, and decide later during modeling whether to:
        - Use them as-is (model net emissions), or
        - Restrict to total_emission > 0 for some models.

Here is a filtered version with strictly positive targets for modeling:

In [89]:
# df_model = df_clean[df_clean['total_emission'] > 0].copy()
# df_model.info()


### 7. Optional: simple range sanity

Quick peek at extremes:

In [90]:
df_clean.describe().T[['min', 'max']]


,min,max
Year,1990.000000,2.020000e+03
Savanna fires,0.000000,1.146164e+05
Forest fires,0.000000,5.222763e+04
Crop Residues,0.000200,3.349007e+04
Rice Cultivation,0.000000,1.649153e+05
Drained organic soils (CO2),0.000000,2.410251e+05
Pesticides Manufacturing,0.000000,1.645900e+04
Food Transport,0.000100,6.794576e+04
Forestland,-797183.079000,1.711211e+05
Net Forest conversion,0.000000,1.605106e+06


Inspect the largest populations to confirm they look reasonable

In [91]:
df_clean[['Area', 'Year', 'Rural population', 'Urban population']].sort_values(
    by='Rural population', ascending=False
).head(10)

,Area,Year,Rural population,Urban population
2989,India,2020,900099113.0,483098640.0
2988,India,2019,896909218.0,471828295.0
2987,India,2018,893272090.0,460779764.0
2986,India,2017,889216746.0,449963381.0
2985,India,2016,884779655.0,439391699.0
2984,India,2015,879984521.0,429069459.0
2983,India,2014,874857621.0,419001673.0
1296,China,1992,871217580.0,359784753.0
1295,China,1991,871145540.0,344764634.0
1297,China,1993,869646991.0,374738726.0


We are looking for truly absurd values (e.g. a population of 1e20).  
For something obviously impossible:

In [92]:
# # Example pattern for unconditional craziness:
# bad_idx = df_clean[df_clean['Rural population'] > 1e10].index
# df_clean = df_clean.drop(index=bad_idx)


*“We have checked for implausible magnitudes, but did not find any that required removal.”*

### 8. Save the cleaned dataset

In [ ]:
df_clean.to_csv("Agrofood_co2_emission_clean.csv", index=False)
